In [1]:
## Import Python libraries ##
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import re
import requests
import scipy.io as sio
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from fuzzywuzzy import process
from iexfinance.stocks import Stock
from IPython.display import Image

In [4]:
## Import the Scientific Beta's Stock Universe (SBSU) data ##
SBSU = pd.read_excel('data/ISS/unvVotingRights-2019Q2.xlsx', sheet_name='Synthese')

## Create additional columns to store related BBG ticker details ##
SBSU['BBG_STOCK_ID'] = np.nan
SBSU['BBG_STOCK_TICKER'] = np.nan
SBSU['BBG_STOCK_TYPE'] = np.nan

## Display the length and first 5 rows of SBSU ##
print('Total no. of stocks in SBSU: %d' % len(SBSU))
SBSU.head()

Total no. of stocks in SBSU: 3017


,ddate,sciBetaId,isin,sedol,MIC,companyName,country,blks,eco,CIQ security SubType Name,securityName,DataScope Voting Rights Per Share,Bloomberg Voting Rights Per Share,diff DSC/BBG,No Voting Rights,BBG_STOCK_ID,BBG_STOCK_TICKER,BBG_STOCK_TYPE
0,2019-04-26,ALT-3675,AT0000606306,B0704T9,XVIE,Raiffeisen Bank International AG,AT,EDE:XE6,D,Common Stock,Bearer Shares,1.0,1.0,NaN,NaN,NaN,NaN,NaN
1,2019-04-26,ETO-1920,AT0000831706,5699373,XVIE,Wienerberger AG,AT,XE6,D,Common Stock,Ordinary Shares,1.0,1.0,NaN,NaN,NaN,NaN,NaN
2,2019-04-26,ETO-9954,AT0000743059,4651459,XVIE,OMV Aktiengesellschaft,AT,EDE:XE6,D,Common Stock,Common Shares,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3,2019-04-26,HAB-5525,AT0000746409,4661607,XVIE,VERBUND AG,AT,XE6,D,Common Stock,Category A Bearer Shares,1.0,1.0,NaN,NaN,NaN,NaN,NaN
4,2019-04-26,MDI-7459,AT0000A18XM4,BPF0548,XVTX,ams AG,AT,XE6,D,Common Stock,Bearer Shares,1.0,1.0,NaN,NaN,NaN,NaN,NaN


In [6]:
## Initialize lists and API details for subsequent calling ##

# Create lists to contain sedol and MIC values #
sedol_values = list(SBSU['sedol'])
mic_values = list(SBSU['MIC'])

# Load API key stored in local directory #
api_keys = pd.read_excel('data/ISS/API_KEYS.xlsx') 
openfigi_key = api_keys['KEY'].loc[api_keys['WEBAPI'] == 'openfigi'][0]

# Create url and header variables to contain API details #
url = 'https://api.openfigi.com/v2/mapping' 
headers = {'Content-Type':'text/json', 'X-OPENFIGI-APIKEY':openfigi_key}

# Initialize counters to track progress #
right_counter = 0
wrong_counter = 0
total_counter = 0

In [147]:
## Trigger API to return related BBG ticker details ##
for i in range(len(sedol_values)):
    # Update payload details #
    payload = '[{"idType":"ID_SEDOL", "idValue": "%s", "micCode": "%s"}]' % (str(sedol_values[i]),str(mic_values[i]))
    # Post request on FIGI website #
    r = requests.post(url, data=payload, headers=headers)

    if 'data' in r.json()[0]:
        for data in r.json()[0]['data']:
            # Update relevant SBSU columns: BBG_STOCK_ID, BBG_STOCK_TICKER, BBG_STOCK_TYPE #
            SBSU.loc[SBSU['sedol'] == sedol_values[i], ['BBG_STOCK_ID']] = r.json()[0]['data'][0]['uniqueID']
            SBSU.loc[SBSU['sedol'] == sedol_values[i], ['BBG_STOCK_TICKER']] = r.json()[0]['data'][0]['ticker']
            SBSU.loc[SBSU['sedol'] == sedol_values[i], ['BBG_STOCK_TYPE']] = r.json()[0]['data'][0]['securityType']
            # Update right_counter #
            right_counter += 1
            break
    else:
        # Update wrong_counter #
        wrong_counter += 1
        
    # Update total_counter #
    total_counter += 1
    
    # Print updated counters after each SBSU row is processed #
    print('Matched: %d, Unmatched: %d, Total: %d' % (right_counter, wrong_counter, total_counter))

## Output the results in an Excel file ##
SBSU.to_excel('data/ISS/SBSU.xlsx', sheet_name='SBSU', index=False)

Matched: 0, Unmatched: 2, Total: 2
Matched: 0, Unmatched: 3, Total: 3
Matched: 0, Unmatched: 4, Total: 4
Matched: 0, Unmatched: 5, Total: 5
Matched: 1, Unmatched: 5, Total: 6
Matched: 1, Unmatched: 6, Total: 7
Matched: 1, Unmatched: 7, Total: 8
Matched: 1, Unmatched: 8, Total: 9
Matched: 1, Unmatched: 9, Total: 10
Matched: 2, Unmatched: 9, Total: 11
Matched: 3, Unmatched: 9, Total: 12
Matched: 4, Unmatched: 9, Total: 13
Matched: 5, Unmatched: 9, Total: 14
Matched: 6, Unmatched: 9, Total: 15
Matched: 7, Unmatched: 9, Total: 16
Matched: 8, Unmatched: 9, Total: 17
Matched: 9, Unmatched: 9, Total: 18
Matched: 10, Unmatched: 9, Total: 19
Matched: 11, Unmatched: 9, Total: 20
Matched: 12, Unmatched: 9, Total: 21
Matched: 13, Unmatched: 9, Total: 22
Matched: 14, Unmatched: 9, Total: 23
Matched: 15, Unmatched: 9, Total: 24
Matched: 16, Unmatched: 9, Total: 25
Matched: 17, Unmatched: 9, Total: 26
Matched: 18, Unmatched: 9, Total: 27
Matched: 19, Unmatched: 9, Total: 28
Matched: 20, Unmatched: 9,

Matched: 209, Unmatched: 9, Total: 218
Matched: 210, Unmatched: 9, Total: 219
Matched: 211, Unmatched: 9, Total: 220
Matched: 212, Unmatched: 9, Total: 221
Matched: 213, Unmatched: 9, Total: 222
Matched: 214, Unmatched: 9, Total: 223
Matched: 215, Unmatched: 9, Total: 224
Matched: 216, Unmatched: 9, Total: 225
Matched: 217, Unmatched: 9, Total: 226
Matched: 218, Unmatched: 9, Total: 227
Matched: 219, Unmatched: 9, Total: 228
Matched: 220, Unmatched: 9, Total: 229
Matched: 221, Unmatched: 9, Total: 230
Matched: 222, Unmatched: 9, Total: 231
Matched: 223, Unmatched: 9, Total: 232
Matched: 224, Unmatched: 9, Total: 233
Matched: 225, Unmatched: 9, Total: 234
Matched: 226, Unmatched: 9, Total: 235
Matched: 227, Unmatched: 9, Total: 236
Matched: 227, Unmatched: 10, Total: 237
Matched: 228, Unmatched: 10, Total: 238
Matched: 229, Unmatched: 10, Total: 239
Matched: 230, Unmatched: 10, Total: 240
Matched: 231, Unmatched: 10, Total: 241
Matched: 232, Unmatched: 10, Total: 242
Matched: 233, Unmat

Matched: 386, Unmatched: 38, Total: 424
Matched: 387, Unmatched: 38, Total: 425
Matched: 388, Unmatched: 38, Total: 426
Matched: 389, Unmatched: 38, Total: 427
Matched: 390, Unmatched: 38, Total: 428
Matched: 391, Unmatched: 38, Total: 429
Matched: 392, Unmatched: 38, Total: 430
Matched: 393, Unmatched: 38, Total: 431
Matched: 394, Unmatched: 38, Total: 432
Matched: 395, Unmatched: 38, Total: 433
Matched: 396, Unmatched: 38, Total: 434
Matched: 397, Unmatched: 38, Total: 435
Matched: 398, Unmatched: 38, Total: 436
Matched: 399, Unmatched: 38, Total: 437
Matched: 400, Unmatched: 38, Total: 438
Matched: 401, Unmatched: 38, Total: 439
Matched: 402, Unmatched: 38, Total: 440
Matched: 403, Unmatched: 38, Total: 441
Matched: 404, Unmatched: 38, Total: 442
Matched: 405, Unmatched: 38, Total: 443
Matched: 406, Unmatched: 38, Total: 444
Matched: 407, Unmatched: 38, Total: 445
Matched: 408, Unmatched: 38, Total: 446
Matched: 409, Unmatched: 38, Total: 447
Matched: 410, Unmatched: 38, Total: 448


Matched: 586, Unmatched: 43, Total: 629
Matched: 587, Unmatched: 43, Total: 630
Matched: 588, Unmatched: 43, Total: 631
Matched: 589, Unmatched: 43, Total: 632
Matched: 590, Unmatched: 43, Total: 633
Matched: 591, Unmatched: 43, Total: 634
Matched: 592, Unmatched: 43, Total: 635
Matched: 593, Unmatched: 43, Total: 636
Matched: 594, Unmatched: 43, Total: 637
Matched: 595, Unmatched: 43, Total: 638
Matched: 596, Unmatched: 43, Total: 639
Matched: 597, Unmatched: 43, Total: 640
Matched: 598, Unmatched: 43, Total: 641
Matched: 599, Unmatched: 43, Total: 642
Matched: 600, Unmatched: 43, Total: 643
Matched: 601, Unmatched: 43, Total: 644
Matched: 602, Unmatched: 43, Total: 645
Matched: 603, Unmatched: 43, Total: 646
Matched: 604, Unmatched: 43, Total: 647
Matched: 605, Unmatched: 43, Total: 648
Matched: 606, Unmatched: 43, Total: 649
Matched: 607, Unmatched: 43, Total: 650
Matched: 608, Unmatched: 43, Total: 651
Matched: 609, Unmatched: 43, Total: 652
Matched: 610, Unmatched: 43, Total: 653


Matched: 760, Unmatched: 74, Total: 834
Matched: 761, Unmatched: 74, Total: 835
Matched: 762, Unmatched: 74, Total: 836
Matched: 763, Unmatched: 74, Total: 837
Matched: 764, Unmatched: 74, Total: 838
Matched: 765, Unmatched: 74, Total: 839
Matched: 766, Unmatched: 74, Total: 840
Matched: 767, Unmatched: 74, Total: 841
Matched: 768, Unmatched: 74, Total: 842
Matched: 769, Unmatched: 74, Total: 843
Matched: 770, Unmatched: 74, Total: 844
Matched: 771, Unmatched: 74, Total: 845
Matched: 772, Unmatched: 74, Total: 846
Matched: 773, Unmatched: 74, Total: 847
Matched: 774, Unmatched: 74, Total: 848
Matched: 775, Unmatched: 74, Total: 849
Matched: 776, Unmatched: 74, Total: 850
Matched: 777, Unmatched: 74, Total: 851
Matched: 778, Unmatched: 74, Total: 852
Matched: 779, Unmatched: 74, Total: 853
Matched: 780, Unmatched: 74, Total: 854
Matched: 781, Unmatched: 74, Total: 855
Matched: 782, Unmatched: 74, Total: 856
Matched: 783, Unmatched: 74, Total: 857
Matched: 784, Unmatched: 74, Total: 858


...

Matched: 2137, Unmatched: 77, Total: 2214
Matched: 2138, Unmatched: 77, Total: 2215
Matched: 2139, Unmatched: 77, Total: 2216
Matched: 2140, Unmatched: 77, Total: 2217
Matched: 2141, Unmatched: 77, Total: 2218
Matched: 2142, Unmatched: 77, Total: 2219
Matched: 2143, Unmatched: 77, Total: 2220
Matched: 2144, Unmatched: 77, Total: 2221
Matched: 2145, Unmatched: 77, Total: 2222
Matched: 2146, Unmatched: 77, Total: 2223
Matched: 2147, Unmatched: 77, Total: 2224
Matched: 2148, Unmatched: 77, Total: 2225
Matched: 2149, Unmatched: 77, Total: 2226
Matched: 2150, Unmatched: 77, Total: 2227
Matched: 2151, Unmatched: 77, Total: 2228
Matched: 2152, Unmatched: 77, Total: 2229
Matched: 2153, Unmatched: 77, Total: 2230
Matched: 2154, Unmatched: 77, Total: 2231
Matched: 2155, Unmatched: 77, Total: 2232
Matched: 2156, Unmatched: 77, Total: 2233
Matched: 2157, Unmatched: 77, Total: 2234
Matched: 2158, Unmatched: 77, Total: 2235
Matched: 2159, Unmatched: 77, Total: 2236
Matched: 2160, Unmatched: 77, Tota

Matched: 2332, Unmatched: 78, Total: 2410
Matched: 2333, Unmatched: 78, Total: 2411
Matched: 2334, Unmatched: 78, Total: 2412
Matched: 2335, Unmatched: 78, Total: 2413
Matched: 2336, Unmatched: 78, Total: 2414
Matched: 2337, Unmatched: 78, Total: 2415
Matched: 2338, Unmatched: 78, Total: 2416
Matched: 2339, Unmatched: 78, Total: 2417
Matched: 2340, Unmatched: 78, Total: 2418
Matched: 2341, Unmatched: 78, Total: 2419
Matched: 2342, Unmatched: 78, Total: 2420
Matched: 2343, Unmatched: 78, Total: 2421
Matched: 2344, Unmatched: 78, Total: 2422
Matched: 2345, Unmatched: 78, Total: 2423
Matched: 2346, Unmatched: 78, Total: 2424
Matched: 2347, Unmatched: 78, Total: 2425
Matched: 2348, Unmatched: 78, Total: 2426
Matched: 2349, Unmatched: 78, Total: 2427
Matched: 2350, Unmatched: 78, Total: 2428
Matched: 2351, Unmatched: 78, Total: 2429
Matched: 2352, Unmatched: 78, Total: 2430
Matched: 2353, Unmatched: 78, Total: 2431
Matched: 2354, Unmatched: 78, Total: 2432
Matched: 2355, Unmatched: 78, Tota

Matched: 2521, Unmatched: 85, Total: 2606
Matched: 2522, Unmatched: 85, Total: 2607
Matched: 2523, Unmatched: 85, Total: 2608
Matched: 2524, Unmatched: 85, Total: 2609
Matched: 2525, Unmatched: 85, Total: 2610
Matched: 2526, Unmatched: 85, Total: 2611
Matched: 2527, Unmatched: 85, Total: 2612
Matched: 2528, Unmatched: 85, Total: 2613
Matched: 2529, Unmatched: 85, Total: 2614
Matched: 2530, Unmatched: 85, Total: 2615
Matched: 2531, Unmatched: 85, Total: 2616
Matched: 2532, Unmatched: 85, Total: 2617
Matched: 2533, Unmatched: 85, Total: 2618
Matched: 2534, Unmatched: 85, Total: 2619
Matched: 2535, Unmatched: 85, Total: 2620
Matched: 2536, Unmatched: 85, Total: 2621
Matched: 2537, Unmatched: 85, Total: 2622
Matched: 2538, Unmatched: 85, Total: 2623
Matched: 2539, Unmatched: 85, Total: 2624
Matched: 2540, Unmatched: 85, Total: 2625
Matched: 2541, Unmatched: 85, Total: 2626
Matched: 2542, Unmatched: 85, Total: 2627
Matched: 2543, Unmatched: 85, Total: 2628
Matched: 2544, Unmatched: 85, Tota

Matched: 2717, Unmatched: 85, Total: 2802
Matched: 2718, Unmatched: 85, Total: 2803
Matched: 2719, Unmatched: 85, Total: 2804
Matched: 2720, Unmatched: 85, Total: 2805
Matched: 2721, Unmatched: 85, Total: 2806
Matched: 2722, Unmatched: 85, Total: 2807
Matched: 2723, Unmatched: 85, Total: 2808
Matched: 2724, Unmatched: 85, Total: 2809
Matched: 2725, Unmatched: 85, Total: 2810
Matched: 2726, Unmatched: 85, Total: 2811
Matched: 2727, Unmatched: 85, Total: 2812
Matched: 2728, Unmatched: 85, Total: 2813
Matched: 2729, Unmatched: 85, Total: 2814
Matched: 2730, Unmatched: 85, Total: 2815
Matched: 2731, Unmatched: 85, Total: 2816
Matched: 2732, Unmatched: 85, Total: 2817
Matched: 2733, Unmatched: 85, Total: 2818
Matched: 2734, Unmatched: 85, Total: 2819
Matched: 2735, Unmatched: 85, Total: 2820
Matched: 2736, Unmatched: 85, Total: 2821
Matched: 2737, Unmatched: 85, Total: 2822
Matched: 2738, Unmatched: 85, Total: 2823
Matched: 2739, Unmatched: 85, Total: 2824
Matched: 2740, Unmatched: 85, Tota

Matched: 2913, Unmatched: 85, Total: 2998
Matched: 2914, Unmatched: 85, Total: 2999
Matched: 2915, Unmatched: 85, Total: 3000
Matched: 2916, Unmatched: 85, Total: 3001
Matched: 2917, Unmatched: 85, Total: 3002
Matched: 2918, Unmatched: 85, Total: 3003
Matched: 2919, Unmatched: 85, Total: 3004
Matched: 2920, Unmatched: 85, Total: 3005
Matched: 2921, Unmatched: 85, Total: 3006
Matched: 2922, Unmatched: 85, Total: 3007
Matched: 2923, Unmatched: 85, Total: 3008
Matched: 2924, Unmatched: 85, Total: 3009
Matched: 2925, Unmatched: 85, Total: 3010
Matched: 2926, Unmatched: 85, Total: 3011
Matched: 2927, Unmatched: 85, Total: 3012
Matched: 2928, Unmatched: 85, Total: 3013
Matched: 2929, Unmatched: 85, Total: 3014
Matched: 2930, Unmatched: 85, Total: 3015
Matched: 2931, Unmatched: 85, Total: 3016
Matched: 2932, Unmatched: 85, Total: 3017
Matched: 2933, Unmatched: 85, Total: 3018


In [166]:
payload = '[{"idType":"ID_SEDOL", "idValue": "2092599", "micCode": "XTSE"}]'
# Post request on FIGI website #
r = requests.post(url, data=payload, headers=headers)
print(r.json())

[{'data': [{'figi': 'BBG000C9K7Q0', 'name': 'BROOKFIELD ASSET MANAGE-CL A', 'ticker': 'BAM/A', 'exchCode': 'CT', 'compositeFIGI': 'BBG000C9K6K8', 'uniqueID': 'EQ0023964300002000', 'securityType': 'Common Stock', 'marketSector': 'Equity', 'shareClassFIGI': 'BBG001SF86D7', 'uniqueIDFutOpt': None, 'securityType2': 'Common Stock', 'securityDescription': 'BAM/A'}]}]
